# Inference Pipeline

### Imports

In [100]:
import pandas as pd
import hopsworks
import joblib
from datetime import datetime
import requests
import numpy as np
from huggingface_hub import notebook_login
from datasets import load_dataset, Dataset
from sklearn import preprocessing as p

### Connect to Hopsworks and download model

In [101]:
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5322
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


### Load LSTM Model from Hopsworks

In [102]:
model = mr.get_model("headlines_sentiment_model", version=4)
model_dir = model.download()
model = joblib.load(model_dir + "/headlines_sentiment_model.pkl")
print(model.summary())

File Name                                             Modified             Size
config.json                                    2023-01-13 17:01:14         2508
metadata.json                                  2023-01-13 17:01:14           64
variables.h5                                   2023-01-13 17:01:14      4751728
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
......embedding
.........vars
............0
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Model: "sequential_6"
_________

### Connect and load batch data from HuggingFace

In [103]:
notebook_login()

# Load scraped batch data from HuggingFace
batch_data = load_dataset("eengel7/sentiment_analysis_batch",split='train')
batch_data = pd.DataFrame(data=batch_data)

2023-01-14 21:05:28,801 WARNING: Using custom data configuration eengel7--sentiment_analysis_batch-8e979c99a41eb6a9
2023-01-14 21:05:28,871 WARNING: Found cached dataset parquet (/Users/evaengel/.cache/huggingface/datasets/eengel7___parquet/eengel7--sentiment_analysis_batch-8e979c99a41eb6a9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


### Predict sentiment of batch data

In [104]:
# Get classification probability for every label
y_predictions = model.predict(batch_data['Headline'].to_list())

1/1 [==============================] - 2s 2s/step


In [97]:
batch_data['Prediction'] = y_predictions.argmax(axis=1)

In [113]:
# Sort batch data depending on classification confidence

batch_data['Confidence'] = np.amax(y_predictions, axis=1)
batch_data = batch_data.sort_values('Confidence', ascending=False)

### Reconstruct dataframe and upload

In [99]:
# Upload to HuggingFace
batch_predictions_dataset = Dataset.from_pandas(batch_data)
batch_predictions_dataset.push_to_hub("eengel7/sentiment_analysis_batch_predictions")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

StopIteration: 